In [1]:
import tensorflow as tf
import os
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator as IDG
from keras import regularizers, optimizers
#Import from keras_preprocessing not from keras.preprocessing

In [2]:

%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:

img_path = './dataset/'

df=pd.DataFrame(columns=("image","lable"))

class_names = sorted(os.listdir(img_path))
for i in class_names:
    for j in os.listdir(img_path + "/" + i):
        df.loc[len(df.index)] = [j,i]  
df.to_csv("csv_file.csv")


In [4]:
import shutil
# Create target Directory if don't exist
if not os.path.exists(img_path + "all"):
    os.mkdir(img_path + "all")
    src_dir = './dataset/'
    # path to destination directory
    dest_dir = './dataset/all'
    # getting all the files in the source directory
    files = os.listdir(src_dir) 
    for i in class_names:
        shutil.copytree(src_dir + i, dest_dir,dirs_exist_ok=True)

In [5]:
# Initialize DataGenerator
train_gen = IDG(rescale=1./255, horizontal_flip=True, rotation_range=20, validation_split=0.2)

# train_images,train_lables,test_images,test_labels = train_test_split(df["image"],df["lable"],test_size=0.3,shuffle=True)
train_images = train_gen.flow_from_dataframe(dataframe=df,directory=img_path + "/" + "all", x_col="image",y_col="lable", class_mode="categorical", subset='training', shuffle=True, batch_size=64,target_size=(64,64))
test_images = train_gen.flow_from_dataframe(dataframe= df,directory=img_path + "/" + "all",x_col="image",y_col="lable", class_mode="categorical", subset='validation', shuffle=True, batch_size=64,target_size=(64,64))


Found 10880 validated image filenames belonging to 12 classes.
Found 2719 validated image filenames belonging to 12 classes.


In [6]:
model = models.Sequential()
model.add(layers.Conv2D(64, (5, 5), padding='same',
                 input_shape=(64,64,3)))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(64, (5, 5)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(128, (5, 5), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(128, (5, 5)))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(1024))
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(11, activation='softmax'))
model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=["accuracy"])

In [7]:

# history=model.fit_generator(generator=train_images,steps_per_epoch=15,validation_steps=15,
#                     validation_data=test_images,
#                     epochs=10
# )

In [8]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
# plt.legend(loc='lower right')
plt.grid(True)

test_loss, test_acc = model.evaluate(test_images, verbose=2)
print(test_acc)

NameError: name 'history' is not defined

In [ ]:
i=1
plt.figure(figsize=(15,20))
for images, labels in train_images:
    id = np.random.randint(len(images))
    image= images[id]
    label=list(labels[id]).index(1)
    pred_label = class_names[np.argmax(model.predict(image[np.newaxis,...]))]
    plt.subplot(5, 4, i)
    plt.imshow(image)
    plt.title(f"Org: {class_names[label]},Pred: {pred_label}")
    plt.axis('off')
    plt.title(f"predic:{pred_label} orig:{class_names[label]}")
    i+=1
    if i>=10: 
        break
